In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

def U_psi(theta):
    """
    Quantum function that generates |psi>, Zenda's state wants to send to Reece.

    Args:
        theta (float): Parameter that generates the state.

    """
    qml.Hadamard(wires = 0)
    qml.CRX(theta, wires = [0,1])
    qml.CRZ(theta, wires = [0,1])

def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """


    Rx = np.array([[np.cos(beta/2), -1j*np.sin(beta/2)], [-1j*np.sin(beta/2), np.cos(beta/2)]])
    Rz = np.array([[np.exp(-1j*alpha/2), 0], [0, np.exp(1j*alpha/2)]])
    Rxz = np.matmul(Rx, Rz)
    op = np.kron(Rxz, Rxz)
    
    num_wires = 2
    # Put your code here #
    dev = qml.device("default.qubit", wires = num_wires)
    
    # op = qml.RX(beta, wires = 0) @ qml.RZ(alpha, wires = 0) @ qml.RX(beta, wires = 1) @ qml.RZ(alpha, wires = 1)
    # print(op)
    @qml.qnode(dev)
    def circuit(theta):
        U_psi(theta)
        return qml.state()
    
    def inner_product(theta):
        psi = circuit(theta)
        # print(psi)
        psi_dg = np.transpose(np.conjugate(psi))
        return np.matmul(psi_dg, np.matmul(op, psi))
    
    def cost(theta):
        # c = -np.abs(circuit(theta))**2
        c = -np.abs(inner_product(theta))**2
        return c

        
#     opt = qml.AdamOptimizer(0.05)
    
#     steps = 100
#     theta = np.random.normal(0, np.pi, 1, requires_grad = True)
#     # print('initial_theta:', theta)
#     for step in range(steps):
#         theta = opt.step(cost, theta)
        
#         # if step % 20 == 0:
#             # print(theta, cost(theta))
    
#     final_cost = -cost(theta)
#     print("Final Cost:", final_cost, "Final theta:", theta)

#     if final_cost >= 1 - epsilon:
#         return True
#     else:
#         return False

    for theta in np.arange(0, 2*np.pi, 0.01):
        if -cost(theta) >= 1-epsilon:
            return True
    return False


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = is_unsafe(*ins)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    
    def bool_to_int(string):
        if string == "True":
            return 1
        return 0

    solution_output = bool_to_int(solution_output)
    expected_output = bool_to_int(expected_output)
    assert solution_output == expected_output, "The solution is not correct."


test_cases = [['[0.1, 0.2, 0.3]', 'True'], ['[1.1, 1.2, 0.3]', 'False'], ['[1.1, 1.2, 0.4]', 'True'], ['[0.5, 1.9, 0.7]', 'True']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")